# TYPHOON protocol showcase

Some of the system packages required for this notebook, including: `ip-route`, `tc`

Use the following cell to configure TYPHOON hyperparameters with environmental variables.

> NB! It should be run *prior* to all the other cells, since hyperparameters are being resolved only once upon `TyphoonCore` class import.

In [ ]:
from os import environ

environ["TYPHOON_MIN_TIMEOUT"] = str(1)
environ["TYPHOON_MAX_TIMEOUT"] = str(4)
environ["TYPHOON_DEFAULT_TIMEOUT"] = str(3)
environ["TYPHOON_MIN_NEXT_IN"] = str(8.0)
environ["TYPHOON_MAX_NEXT_IN"] = str(32.0)
environ["TYPHOON_MAX_RETRIES"] = str(3)

This notebook requires some extra capabilities being applied to your `python` executable and also added to the `JuPyter` server process.
Don't worry: everything can be done just in a few steps.

> Why are these capabilities needed?
> Well, let's say for "safety" reasons.
> We do not want to apply **tc** (traffic control) commands directly to the loopback interface, right?
> That's why we need a pair of virtual interfaces with virtual IPs (`veth`s), so that we can apply fine-grained filters.
> And creating these interfaces requires some permissions.

First of all, the following cell should be executed before any other notebook cells (it might throw an error!):

In [ ]:
from pathlib import Path
from sys import path

path.append(str(Path() / ".."))

from typhoon.utils import configure_ambient_permissions

configure_ambient_permissions()

If the previous cell has thrown an error, that means that your `python` executable does not have `CAP_NET_ADMIN` and `CAP_NET_RAW` right now.
This can be solved by either of these three solutions:

1. Run the server in a `Docker` container (either `privileged` or with `CAP_NET_ADMIN` set) - the dockerfile is *not* provided.
2. Run the following cell to patch your current python executable (don't forget to run the next cell when you're done!).
3. Run the server with `sudo` (might be dangerous and not recommended!).

Whatever way you have chosen, the kernel should be restarted.

In [ ]:
from typhoon.utils import toggle_network_permissions

toggle_network_permissions(True)

Run this cell after you finished working with the notebook - it will unpatch you `python` executable.

In [ ]:
from typhoon.utils import toggle_network_permissions

toggle_network_permissions(False)

## Ideal connection

First and foremost, TYPHOON is designed to be **random**.
That includes:

1. Randomly-looking packet contents.
2. Random packet lengths.
3. Random delays between packets.

#### Packet contents

Packet content randomness is achieved by encryption.
Packets are encrypted completely, TYPHOON uses distinct keys for host-port pairs (provided by UDP header).
That is the baseline, some other hiding techniques might be added later (if needed).

#### Packet lengths

Packet length randomness is achieved by attaching a random tail to every package.
Packet length will be displayed in bar charts below as *bar heights*.

#### Packet delays

Packet delays randomness is achieved by applying a random timeout between service packets.
NB! This timeout is not applied to data packets, so in examples below you will usually see them as pairs.

Simple sanity check here.
Client connects to echo server, sends it a small package, receives response, checks it's the same and exits.

First two packets should be initialization packets, then we expect to observe a shadowride request packet (data + handshake) and a data response packet.
Last two packets should be termination packets.

As a result, a small sequence diagram is produced, showing packet exchange, their direction, size and timestamp.

In [ ]:
from sources.protocol.typhoon_socket import TyphoonListener, TyphoonClient
from sources.protocol.utils import ProtocolReturnCode

from typhoon.utils import GraphConf, sniff, LISTENER_KEY, CLIENT_KEY, USER_TOKEN


async def echo_server_callback(user_id: int, data: bytes) -> bytes:
    print(f"Received data from client {user_id}: {data}")
    return data


async def success_listener_callback(name: str, token: bytes) -> ProtocolReturnCode:
    print(f"Initialized client '{name}' with token: {token}")
    return ProtocolReturnCode.SUCCESS


async with sniff(GraphConf()) as s:
    async with TyphoonListener(LISTENER_KEY, s.server_address).ctx(success_listener_callback, echo_server_callback) as l:
        async with TyphoonClient(CLIENT_KEY, USER_TOKEN, s.server_address, l.port, s.client_address).ctx() as c:
            request = b"Hi server!"
            await c.write(request)
            response = await c.read()
            assert(request == response)

Stale connection is shown here.
Client is connected to server, waits for some time and disappears.
Server tries to restore connection, but fails and closes it after some time.

First two packets should be initialization packets, then several successful handshakes should happen.
Finally, some unsuccessful handshake attempts by server should be present - and finally a termination packet.

As a result, a small packet plot is produced, showing packet exchange, their type, size and timestamp.

In [ ]:
from asyncio import sleep
from os import environ

from sources.protocol.typhoon_socket import TyphoonListener, TyphoonClient

from typhoon.utils import PlotConf, sniff, LISTENER_KEY, CLIENT_KEY, USER_TOKEN

SLEEP_DELAY = (float(environ["TYPHOON_MAX_NEXT_IN"]) + float(environ["TYPHOON_MAX_TIMEOUT"])) * 5
TIMEOUT_DELAY = float(environ["TYPHOON_MAX_NEXT_IN"]) * (float(environ["TYPHOON_MAX_RETRIES"]) + 1)


async with sniff(PlotConf()) as s:
    async with TyphoonListener(LISTENER_KEY, s.server_address).ctx(data_callback=lambda _: None) as l:
        async with TyphoonClient(CLIENT_KEY, USER_TOKEN, s.server_address, l.port, s.client_address).ctx(lambda _: None, False) as c:
            print(f"Connection established! Sleeping for {SLEEP_DELAY}...")
            await sleep(SLEEP_DELAY)
        print(f"Connection interrupted! Waiting for {TIMEOUT_DELAY}...")
        await sleep(TIMEOUT_DELAY)

Successful connection is shown here.
Client is connected to server, they exchange data packets every second.
In the end, the connection is terminated successfully.

First two packets should be initialization packets, last two packets should be termination packets.
In between, multiple data and shadowride packages should be observable.
Because of the frequent data exchange, no pure handshake packets should be there.

As a result, a small packet plot is produced, showing packet exchange, their type and size.
The receiving time is omitted.

In [ ]:
from sources.protocol.typhoon_socket import TyphoonListener, TyphoonClient

from typhoon.utils import PlotConf, sniff, SequenceConfig, LISTENER_KEY, CLIENT_KEY, USER_TOKEN

CONFIG = SequenceConfig(messages=96, quiet=True, client_sequence=lambda _: 1, server_sequence=lambda _: 0)


async with sniff(PlotConf(expand_time=False)) as s:
    async with TyphoonListener(LISTENER_KEY, s.server_address).ctx(data_callback=CONFIG.seq_server_callback, log_level=10) as l:
        async with TyphoonClient(CLIENT_KEY, USER_TOKEN, s.server_address, l.port, s.client_address).ctx(CONFIG.seq_client_callback, log_level=10) as c:
            await CONFIG.process(c)

## Network in chaos

normal

In [ ]:
from sources.protocol.typhoon_socket import TyphoonListener, TyphoonClient

from typhoon.utils import PlotConf, sniff, SequenceConfig, LISTENER_KEY, CLIENT_KEY, USER_TOKEN

CONFIG = SequenceConfig()


async with sniff(PlotConf()) as s:
    async with TyphoonListener(LISTENER_KEY, s.server_address).ctx(data_callback=CONFIG.seq_server_callback, log_level=10) as l:
        async with TyphoonClient(CLIENT_KEY, USER_TOKEN, s.server_address, l.port, s.client_address).ctx(CONFIG.seq_client_callback, log_level=10) as c:
            await CONFIG.process(c)

jitter

In [ ]:
from asyncio import sleep

from sources.protocol.typhoon_socket import TyphoonListener, TyphoonClient

from typhoon.utils import PlotConf, sniff, SequenceConfig, LISTENER_KEY, CLIENT_KEY, USER_TOKEN

CONFIG = SequenceConfig()


async with sniff(PlotConf(), r"delay 100ms 75ms") as s:
    async with TyphoonListener(LISTENER_KEY, s.server_address).ctx(data_callback=CONFIG.seq_server_callback, log_level=10) as l:
        async with TyphoonClient(CLIENT_KEY, USER_TOKEN, s.server_address, l.port, s.client_address).ctx(CONFIG.seq_client_callback, log_level=10) as c:
            await CONFIG.process(c)
            await sleep(CONFIG.base_delay() * 4)

drop

> WARNING! The following cell might fail in approximately 3% of the runs!

In [ ]:
from sources.protocol.typhoon_socket import TyphoonListener, TyphoonClient

from typhoon.utils import PlotConf, sniff, SequenceConfig, LISTENER_KEY, CLIENT_KEY, USER_TOKEN

CONFIG = SequenceConfig()


async with sniff(PlotConf(), r"loss 30%") as s:
    async with TyphoonListener(LISTENER_KEY, s.server_address).ctx(data_callback=CONFIG.seq_server_callback, log_level=10) as l:
        async with TyphoonClient(CLIENT_KEY, USER_TOKEN, s.server_address, l.port, s.client_address).ctx(CONFIG.seq_client_callback, log_level=10) as c:
            await CONFIG.process(c)

duplicate

In [ ]:
from sources.protocol.typhoon_socket import TyphoonListener, TyphoonClient

from typhoon.utils import PlotConf, sniff, SequenceConfig, LISTENER_KEY, CLIENT_KEY, USER_TOKEN

CONFIG = SequenceConfig()


async with sniff(PlotConf(), r"duplicate 30%") as s:
    async with TyphoonListener(LISTENER_KEY, s.server_address).ctx(data_callback=CONFIG.seq_server_callback, log_level=10) as l:
        async with TyphoonClient(CLIENT_KEY, USER_TOKEN, s.server_address, l.port, s.client_address).ctx(CONFIG.seq_client_callback, log_level=10) as c:
            await CONFIG.process(c)

reorder

In [ ]:
from asyncio import sleep

from sources.protocol.typhoon_socket import TyphoonListener, TyphoonClient

from typhoon.utils import PlotConf, sniff, SequenceConfig, LISTENER_KEY, CLIENT_KEY, USER_TOKEN

CONFIG = SequenceConfig()


async with sniff(PlotConf(), r"delay 125ms reorder 30% 50%") as s:
    async with TyphoonListener(LISTENER_KEY, s.server_address).ctx(data_callback=CONFIG.seq_server_callback, log_level=10) as l:
        async with TyphoonClient(CLIENT_KEY, USER_TOKEN, s.server_address, l.port, s.client_address).ctx(CONFIG.seq_client_callback, log_level=10) as c:
            await CONFIG.process(c)
            await sleep(CONFIG.base_delay() * 2.5)

corrupt

In [ ]:
from sources.protocol.typhoon_socket import TyphoonListener, TyphoonClient

from typhoon.utils import PlotConf, sniff, SequenceConfig, LISTENER_KEY, CLIENT_KEY, USER_TOKEN

CONFIG = SequenceConfig(quiet=True)


async with sniff(PlotConf(), r"corrupt 30%") as s:
    async with TyphoonListener(LISTENER_KEY, s.server_address).ctx(data_callback=CONFIG.seq_server_callback, log_level=10) as l:
        async with TyphoonClient(CLIENT_KEY, USER_TOKEN, s.server_address, l.port, s.client_address).ctx(CONFIG.seq_client_callback, log_level=10) as c:
            await CONFIG.process(c)